**개요 : bert 를 사용해 음식을 추천해주는 코드를 작성했다.**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MachineLearningPractice/Colab-Notebooks/FarmKit/foodfile/food2.csv')

In [3]:
df.head()

,구분,음식명,영문,음식설명
0,면,가락국수,Thick Noodles in Clear Broth,"Noodles in a clear broth, topped with vegetabl..."
1,떡,가래떡,Cylindrical Rice Pasta,Cylindrical rice pasta sliced diagonally into ...
2,찜,가오리찜,Steamed Stingray,Marinated stingray steamed and served with thi...
3,어류,가자미,Sole,NaN
4,구이,가자미구이,Grilled Sole,"Sole seasoned with salt then grilled, often to..."


**결손 데이터 처리**

In [4]:
df = df.dropna()

In [5]:
df.head()

,구분,음식명,영문,음식설명
0,면,가락국수,Thick Noodles in Clear Broth,"Noodles in a clear broth, topped with vegetabl..."
1,떡,가래떡,Cylindrical Rice Pasta,Cylindrical rice pasta sliced diagonally into ...
2,찜,가오리찜,Steamed Stingray,Marinated stingray steamed and served with thi...
4,구이,가자미구이,Grilled Sole,"Sole seasoned with salt then grilled, often to..."
5,젓갈,가자미식해,Spicy Fermented Sole,A spicy dish of sole mixed with cooked millet ...


In [6]:
df.reset_index(inplace=True)
df.drop('index', inplace=True, axis=1)
df.head(10)

,구분,음식명,영문,음식설명
0,면,가락국수,Thick Noodles in Clear Broth,"Noodles in a clear broth, topped with vegetabl..."
1,떡,가래떡,Cylindrical Rice Pasta,Cylindrical rice pasta sliced diagonally into ...
2,찜,가오리찜,Steamed Stingray,Marinated stingray steamed and served with thi...
3,구이,가자미구이,Grilled Sole,"Sole seasoned with salt then grilled, often to..."
4,젓갈,가자미식해,Spicy Fermented Sole,A spicy dish of sole mixed with cooked millet ...
5,조림,가자미조림,Braised Sole,"Sole seasoned with soy sauce, sugar, green oni..."
6,나물,가지나물,Eggplant Namul,"Eggplant steamed then marinated in soy sauce, ..."
7,선,가지선,Stuffed Eggplant,"Eggplants cut lengthwise, filled with seasonin..."
8,"전,적",가지전,Pan-fried Eggplant,"Eggplant slices coated with flour, dipped in e..."
9,찜,가지찜,Stuffed Eggplant with Beef,"Eggplant cut lengthwise, filled with seasoned ..."


**bert 사용**

In [7]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 43.7 MB/s 
     |████████████████████████████████| 596 kB 46.0 MB/s 
     |████████████████████████████████| 101 kB 8.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
import torch
from transformers import AutoModel, AutoTokenizer

MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
# [CLS] token을 얻기 위한 함수
def get_cls_token(sent_A):
    model.eval()
    tokenized_sent = tokenizer(
            sent_A,
            return_tensors="pt",
            truncation=True,
            add_special_tokens=True,
            max_length=128
    )
    with torch.no_grad():# 그라디엔트 계산 비활성화
        outputs = model(    # **tokenized_sent
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
            )
    logits = outputs.last_hidden_state[:,0,:].detach().cpu().numpy()
    return logits

In [10]:
# query [CLS] token hidden 확인

print("=========================== 음식 설명")
query = df.loc[6, '음식설명']
print(query)

print("\n=========================== cls token")
query_cls_hidden = get_cls_token(query)
print(query_cls_hidden)
print(query_cls_hidden.shape)

=========================== 음식 설명
Eggplant steamed then marinated in soy sauce, green onions, garlic, red chili pepper powder, sesame oil and vinegar.

=========================== cls token
[[ 5.29868416e-02 -1.08209431e-01 -6.38060793e-02  1.27211720e-01
   1.16120744e+00  9.62023586e-02 -1.72047094e-02  5.98862588e-01
  -5.58549426e-02  4.88000095e-01 -4.53954816e-01  6.31934106e-01
  -4.68346447e-01 -4.09165978e-01 -7.88309395e-01 -4.46759909e-01
  -5.39088607e-01  8.14099535e-02  1.01804703e-01  1.15845844e-01
   6.15962803e-01 -5.10273457e-01 -3.19566011e-01  3.33208650e-01
  -1.85395300e-01 -2.35453188e-01 -2.53399909e-01 -3.40018690e-01
   1.18877482e+00 -6.13792002e-01  3.74996275e-01  4.20847356e-01
   2.70652086e-01 -4.91284043e-01 -1.99456751e-01  1.29448086e-01
  -5.89994073e-01  4.32801425e-01 -3.04050714e-01  1.24511905e-01
   1.16690360e-01  5.28950512e-01 -2.80295551e-01 -5.85184491e-04
   9.97224823e-02  4.23391521e-01  7.56340176e-02 -3.59821320e-01
   8.67069364e-01 

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
def cls_token_array(df, column) :
  dataset_cls_hidden = []
  for i in range(df.shape[0]):
      q_cls = get_cls_token(str(df.loc[i, column])) #column 예: '음식설명', '영문'
      dataset_cls_hidden.append(q_cls)
  dataset_cls_hidden = np.array(dataset_cls_hidden).squeeze(axis=1)
  #print(dataset_cls_hidden)   
  print(dataset_cls_hidden.shape) # column에 대한 [CLS] 토큰 벡터
  return dataset_cls_hidden

column = '음식설명'
dataset_cls_hidden = cls_token_array(df, column)

(691, 768)


In [13]:
def food_recommendation(df, dataset_cls_hidden, column, food): 
  index = df[df['음식명'] == food].index[0]
  food_cls_hidden = get_cls_token(str(df.loc[index, column]))

  cos_sim = cosine_similarity(food_cls_hidden, dataset_cls_hidden)
  #print(cos_sim)

  top = np.argpartition(cos_sim, -10)[:, -10:] # 상위 10개 인덱스 추출

  return df.iloc[top[0]]['음식명']

**피쳐 벡터화 후 코사인 유사도를 쓴 결과와 비교**

In [14]:
food_recommendation(df, dataset_cls_hidden, column, '가락국수')

370       송이찜
268     민어매운탕
388       숙실과
181     동태매운탕
304    복(어)지리
520       잡산적
398       시래기
374     쇠고기산적
8         가지전
0        가락국수
Name: 음식명, dtype: object

In [15]:
food_recommendation(df, dataset_cls_hidden, column, '된장찌개')

284        배추속댓국
195          된장국
182        동태섞박지
569           증편
285          배춧국
139          녹두죽
486    우거짓국/우거지탕
485          용봉탕
419          아욱국
196         된장찌개
Name: 음식명, dtype: object

In [16]:
food_recommendation(df, dataset_cls_hidden, column, '가지찜')

8        가지전
164    도라지생채
343     새우튀김
565      죽순찜
9        가지찜
7        가지선
680      홍합초
617     콩팥구이
290      뱅어포
381       수란
Name: 음식명, dtype: object

In [17]:
food_recommendation(df, dataset_cls_hidden, column, '약과')

429      양곱창구이
662        해삼탕
387        수제비
100     깨(엿)강정
333         산자
65          곶감
426    약밥 / 약식
38          경단
510        인절미
425         약과
Name: 음식명, dtype: object

**영윤 bert와 비교**

In [18]:
food_recommendation(df, dataset_cls_hidden, column, '감자볶음')

31        겉절이
89      김치누름적
651    한치(물)회
171     도토리냉면
233       명란젓
147      다시마쌈
361       소라회
2        가오리찜
228      메밀국수
19       감자볶음
Name: 음식명, dtype: object

In [19]:
food_recommendation(df, dataset_cls_hidden, column, '칼국수')

670       호박범벅
73        국수장국
64        곱창전골
602        칼국수
511       임자수탕
163      도라지나물
499        육개장
120        꿩만두
229    메밀묵(무침)
232       멸치볶음
Name: 음식명, dtype: object

In [20]:
food_recommendation(df, dataset_cls_hidden, column, '고구마튀김')

362        소면
170       도미회
377     쇠고기튀김
83        김구이
86        김부각
651    한치(물)회
3       가자미구이
512       잉어찜
43      고구마튀김
67        광어회
Name: 음식명, dtype: object

In [21]:
food_recommendation(df, dataset_cls_hidden, column, '된장국')

418      아귀탕/아구탕
132     냉이나물(무침)
656       해물된장찌개
157         더덕김치
297         보리수단
284        배추속댓국
283        배추꼬릿국
285          배춧국
195          된장국
486    우거짓국/우거지탕
Name: 음식명, dtype: object